In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [7]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
#     aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
#                                                                       labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ ~s', Q = ~env(s, a)
    next_states_logits, Qs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                 action_size=action_size, state_size=state_size)
    next_states_logits_, Qs_ = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, # GQN
                                                                   labels=next_states_labels))
#     eloss = -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits_, # GQN
#                                                                     labels=next_states_labels)) # max loss/entropy
#     aloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits_, # GQN
#                                                                    labels=next_states_labels)) # min loss/entropy
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits_, # GQN
                                                                    labels=next_states_labels)) # min loss/entropy
    aloss = -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits_, # GQN
                                                                    labels=next_states_labels)) # max loss/entropy
    ##############################################################
    Qlogits = tf.reshape(Qs, shape=[-1])
    Qlogits_ = tf.reshape(Qs_, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits_, # GAN
                                                                    labels=tf.zeros_like(rates))) # min result/rate
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits_, # GAN
                                                                    labels=tf.ones_like(rates))) # max result/rate
    return actions_logits, aloss, eloss

In [8]:
def model_opt(a_loss, e_loss, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
    return a_opt, e_opt

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt = model_opt(a_loss=self.a_loss, e_loss=self.e_loss,
                                           a_learning_rate=a_learning_rate,
                                           e_learning_rate=e_learning_rate)

In [10]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
        return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [11]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [12]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [13]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [14]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [15]:
def minibatch(memory):
    batch = memory.buffer
    #for idx in range(memory_size// batch_size):
    while True:
        idx = np.random.choice(np.arange(memory_size// batch_size))
        states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        states = states[rates >= np.max(rates)]
        actions = actions[rates >= np.max(rates)]
        next_states = next_states[rates >= np.max(rates)]
        rewards = rewards[rates >= np.max(rates)]
        dones = dones[rates >= np.max(rates)]
        rates = rates[rates >= np.max(rates)]
        if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
            break
    return states, actions, next_states, rewards, dones, rates

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        rate = -1
        
        # Episode steps
        for num_step in range(1111111111111):
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            total_reward += reward
            state = next_state
            
            # Online Training/on-policy
            states, actions, next_states, rewards, dones, rates = minibatch(memory=memory)
            feed_dict = {model.states: states, model.actions: actions, model.next_states: next_states,
                         model.rewards: rewards, model.dones: dones, model.rates: rates}
            eloss, _ = sess.run([model.e_loss, model.e_opt], feed_dict)
            aloss, _ = sess.run([model.a_loss, model.a_opt], feed_dict)
            eloss_batch.append(eloss)
            aloss_batch.append(aloss)
            if done is True:
                break
                
        # Offline Training/off-policy
        # EMPTYYYYYYYYYYYYY#####
        # EMPTYYYYYYYYYYYYY#####
        # EMPTYYYYYYYYYYYYY#####
        # EMPTYYYYYYYYYYYYY#####
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:27.0000 R:27.0000 rate:0.0540 aloss:0.0040 eloss:2.7924
Episode:1 meanR:35.0000 R:43.0000 rate:0.0860 aloss:0.0067 eloss:2.7827
Episode:2 meanR:35.0000 R:35.0000 rate:0.0700 aloss:0.0073 eloss:2.7740
Episode:3 meanR:36.0000 R:39.0000 rate:0.0780 aloss:0.0080 eloss:2.7695
Episode:4 meanR:40.8000 R:60.0000 rate:0.1200 aloss:0.0023 eloss:2.7647
Episode:5 meanR:36.3333 R:14.0000 rate:0.0280 aloss:0.0075 eloss:2.7582
Episode:6 meanR:34.8571 R:26.0000 rate:0.0520 aloss:0.0082 eloss:2.7513
Episode:7 meanR:34.1250 R:29.0000 rate:0.0580 aloss:0.0088 eloss:2.7495
Episode:8 meanR:32.3333 R:18.0000 rate:0.0360 aloss:0.0104 eloss:2.7483
Episode:9 meanR:30.6000 R:15.0000 rate:0.0300 aloss:0.0061 eloss:2.7470
Episode:10 meanR:30.3636 R:28.0000 rate:0.0560 aloss:0.0069 eloss:2.7438
Episode:11 meanR:29.2500 R:17.0000 rate:0.0340 aloss:0.0058 eloss:2.7419
Episode:12 meanR:28.3077 R:17.0000 rate:0.0340 aloss:0.0079 eloss:2.7380
Episode:13 meanR:27.5714 R:18.0000 rate:0.0360 aloss:0.0058 e

Episode:113 meanR:15.8400 R:25.0000 rate:0.0500 aloss:0.0496 eloss:2.5293
Episode:114 meanR:16.2300 R:61.0000 rate:0.1220 aloss:0.0487 eloss:2.5324
Episode:115 meanR:16.4400 R:33.0000 rate:0.0660 aloss:0.0456 eloss:2.5324
Episode:116 meanR:16.9000 R:58.0000 rate:0.1160 aloss:0.0535 eloss:2.5238
Episode:117 meanR:17.1000 R:36.0000 rate:0.0720 aloss:0.0632 eloss:2.4964
Episode:118 meanR:17.6800 R:74.0000 rate:0.1480 aloss:0.0710 eloss:2.4902
Episode:119 meanR:17.9300 R:39.0000 rate:0.0780 aloss:0.0564 eloss:2.5152
Episode:120 meanR:18.2200 R:40.0000 rate:0.0800 aloss:0.0688 eloss:2.4905
Episode:121 meanR:18.5600 R:48.0000 rate:0.0960 aloss:0.0768 eloss:2.4857
Episode:122 meanR:19.0000 R:58.0000 rate:0.1160 aloss:0.0901 eloss:2.4695
Episode:123 meanR:19.3100 R:43.0000 rate:0.0860 aloss:0.0952 eloss:2.4667
Episode:124 meanR:19.6200 R:45.0000 rate:0.0900 aloss:0.1032 eloss:2.4557
Episode:125 meanR:19.9300 R:42.0000 rate:0.0840 aloss:0.1054 eloss:2.4444
Episode:126 meanR:20.7200 R:93.0000 ra

Episode:224 meanR:17.9600 R:10.0000 rate:0.0200 aloss:0.4739 eloss:2.1962
Episode:225 meanR:17.6300 R:9.0000 rate:0.0180 aloss:0.5132 eloss:2.1840
Episode:226 meanR:16.8000 R:10.0000 rate:0.0200 aloss:0.4943 eloss:2.2190
Episode:227 meanR:16.4000 R:8.0000 rate:0.0160 aloss:0.4925 eloss:2.1902
Episode:228 meanR:16.0600 R:10.0000 rate:0.0200 aloss:0.5031 eloss:2.1736
Episode:229 meanR:15.6000 R:10.0000 rate:0.0200 aloss:0.4763 eloss:2.1498
Episode:230 meanR:15.1900 R:10.0000 rate:0.0200 aloss:0.5250 eloss:2.1999
Episode:231 meanR:14.5600 R:10.0000 rate:0.0200 aloss:0.5180 eloss:2.1939
Episode:232 meanR:14.2700 R:8.0000 rate:0.0160 aloss:0.4815 eloss:2.2256
Episode:233 meanR:13.9200 R:10.0000 rate:0.0200 aloss:0.5403 eloss:2.1874
Episode:234 meanR:13.7500 R:10.0000 rate:0.0200 aloss:0.5070 eloss:2.2108
Episode:235 meanR:13.5200 R:8.0000 rate:0.0160 aloss:0.5348 eloss:2.1800
Episode:236 meanR:13.2900 R:9.0000 rate:0.0180 aloss:0.5340 eloss:2.1777
Episode:237 meanR:13.0400 R:10.0000 rate:0.

Episode:337 meanR:9.4300 R:9.0000 rate:0.0180 aloss:1.1768 eloss:1.9838
Episode:338 meanR:9.4100 R:8.0000 rate:0.0160 aloss:1.4581 eloss:1.9492
Episode:339 meanR:9.4100 R:9.0000 rate:0.0180 aloss:1.3782 eloss:1.9359
Episode:340 meanR:9.3900 R:8.0000 rate:0.0160 aloss:1.4033 eloss:1.9887
Episode:341 meanR:9.4100 R:11.0000 rate:0.0220 aloss:1.2963 eloss:1.9727
Episode:342 meanR:9.3900 R:8.0000 rate:0.0160 aloss:1.3272 eloss:1.9733
Episode:343 meanR:9.3900 R:10.0000 rate:0.0200 aloss:1.4277 eloss:1.9664
Episode:344 meanR:9.3900 R:10.0000 rate:0.0200 aloss:1.4632 eloss:1.9550
Episode:345 meanR:9.3800 R:9.0000 rate:0.0180 aloss:1.4812 eloss:1.9518
Episode:346 meanR:9.3700 R:9.0000 rate:0.0180 aloss:1.4404 eloss:1.9560
Episode:347 meanR:9.3800 R:10.0000 rate:0.0200 aloss:1.3933 eloss:1.9630
Episode:348 meanR:9.3800 R:10.0000 rate:0.0200 aloss:1.4547 eloss:1.9171
Episode:349 meanR:9.3700 R:9.0000 rate:0.0180 aloss:1.4541 eloss:1.9491
Episode:350 meanR:9.3700 R:10.0000 rate:0.0200 aloss:1.4641

Episode:451 meanR:9.5900 R:13.0000 rate:0.0260 aloss:2.0434 eloss:1.8607
Episode:452 meanR:9.6300 R:12.0000 rate:0.0240 aloss:1.9831 eloss:1.9024
Episode:453 meanR:9.6900 R:16.0000 rate:0.0320 aloss:2.0484 eloss:1.8834
Episode:454 meanR:9.7400 R:14.0000 rate:0.0280 aloss:1.9738 eloss:1.8790
Episode:455 meanR:9.7900 R:15.0000 rate:0.0300 aloss:2.0246 eloss:1.8558
Episode:456 meanR:9.7900 R:11.0000 rate:0.0220 aloss:1.8587 eloss:1.9164
Episode:457 meanR:9.8100 R:12.0000 rate:0.0240 aloss:1.9812 eloss:1.8917
Episode:458 meanR:9.8200 R:11.0000 rate:0.0220 aloss:1.8776 eloss:1.8810
Episode:459 meanR:9.8900 R:16.0000 rate:0.0320 aloss:1.8800 eloss:1.8663
Episode:460 meanR:9.9800 R:18.0000 rate:0.0360 aloss:1.9677 eloss:1.8920
Episode:461 meanR:10.0500 R:15.0000 rate:0.0300 aloss:1.8851 eloss:1.8670
Episode:462 meanR:10.0900 R:13.0000 rate:0.0260 aloss:1.9685 eloss:1.8625
Episode:463 meanR:10.1400 R:15.0000 rate:0.0300 aloss:1.9402 eloss:1.8889
Episode:464 meanR:10.2300 R:19.0000 rate:0.0380 

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
